In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler


In [ ]:
import sys, argparse, logging
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os
import pickle

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed
from tmqrfeed.manager import DataManager
from tmqrfeed.position import Position
from tmqr.errors import *
from tmqrfeed.costs import Costs

In [ ]:
dm = DataManager()
feed = dm.datafeed

In [ ]:
from tmqrfeed.quotes.quote_contfut import QuoteContFut

In [ ]:
dm.series_primary_set(QuoteContFut, 'US.CL',
                                   timeframe='D', decision_time_shift=5)

dm.costs_set('US', Costs() )

In [ ]:
ohlc = dm.quotes()

In [ ]:

pos = Position(dm)

for dt in ohlc.index[1000:]:
    #print(dt)
    try:
        fut, opt_chain = dm.chains_options_get("US.CL", dt)
        pos.keep_previous_position(dt)
        
        if pos.almost_expired_ratio(dt) == 1:
            pos.close(dt)
        
        if not pos.has_position(dt):
            pos.add_transaction(dt, opt_chain.find(dt, 0, 'C'), 1.0)
        
    except ChainNotFoundError:
        print(f"ChainNotFoundError: {dt}")
        pass
    #except QuoteNotFoundError:
    #    print(f"QuoteNotFoundError: {dt}")
    

In [ ]:
pos

In [ ]:
pos_pnl = pos.get_pnl_series()

In [ ]:
pos_pnl['equity_execution'].plot()

In [ ]:
ohlc.c.plot()